# The  Research  Space

"Here we use a large dataset of scholarly publications disambiguated at the individual level to create a map of science — or research  space — where links connect pairs of fields based on the probability that an individual has published in both of them."
[Original Article](https://arxiv.org/ftp/arxiv/papers/1602/1602.08409.pdf)

[Artigo suplementar](https://link.springer.com/content/pdf/10.1140/epjds/s13688-019-0210-z.pdf)

In [ ]:
import pandas as pd
import numpy as np
import re

### Our data

PRECISO CHECAR O PARSER NOVAMENTE. ALGUNS ANOS APARECERAM COMO 'rint' OU 'onic'. E EM ALGUNS ARTIGOS O NÚMERO DE AUTORES FOI ZERO.

Todas as publicações

In [ ]:
articles = pd.read_csv("../dataset/lattes_categories.csv", sep=";sep;", engine="python")
articles = articles[(articles["ano"] != 'rint') & (articles["ano"] != 'onic')]
articles.head()

### Research Space

" $\phi_{ff'}(T)$ is  the  adjacency  matrix  representing  the  research  space expressed  by  the  career trajectory of scientists in our dataset observed up to time T. "

Get all the categories from an article

In [ ]:
def catg(s):
    return [re.sub(r"\s?\(Q[1-9]\)", "", x).strip().lower() for x in s.split(";")]

Create a dict to represent the X matrix

In [ ]:
def X(t):
    x = dict()
    for _, row in articles.iterrows():
        if int(row["ano"]) < t:
            fs = catg(row["catg"])
            nf = len(fs)
            for field in fs:
                
                if row["num"] == 0:
                    continue
                
                if (row["pesq"], field) in x:
                    x[(row["pesq"], field)] += 1/(nf * row["num"])
                else:
                    x[(row["pesq"], field)] = 1/(nf * row["num"])
    
    print("X done")
    return x

Create a dict to represent the P matrix

In [ ]:
def P(t):
    x = X(t)
    p = dict()
    
    for sf in x:
        if x[sf] > 0.1:
            p[sf] = 1
            
    print("P done")
    return [p, x]

Create the M matrix

In [ ]:
def M(t):
    p, x = P(t)
    s = set()
    f = set()
    
    for sf in p:
        s.add(sf[0])
        f.add(sf[1])
        
    of = sorted(list(f))
    indices = {u: v for v, u in enumerate(of)}
    n = len(of)
    m = np.zeros((n,n))
    
    for i in range(len(of)):
        for j in range(i+1, len(of)):
            for k in s:
                if (k,of[i]) in p and (k,of[j]) in p:
                    m[i,j] += 1
                    m[j,i] += 1
                    
    print("M done")
    return [m, p, of, x]

Create the phi matrix

In [ ]:
def phi(t):
    m, p, of, x = M(t)
    indices = dict()
    sums = np.zeros(len(m))
    
    for sf in p:
        if sf[1] not in indices:
            indices[sf[1]] = of.index(sf[1])
        sums[indices[sf[1]]] += 1

    phi = m.copy()
    for i in range(len(m)):
        phi[:,i] /= sums[i]
    
    print("phi done")
    return [phi, of, x]

Salvar só pra ter certeza né. As etapas do X e do M são bastante demoradas.

In [ ]:
k, of, x = phi(2011)
np.save("../dataset/phi_matrix_2011.npy", k)
np.save("../dataset/x_dict_2011.npy", x)

with open("../dataset/of_2011.txt", "w") as f:
    for item in of:
        f.write("{}\n".format(item))